In [10]:
# --------------------- Header -----------------
import numpy as np
import make_data as md
import pickle
import matplotlib.pyplot as plt
import os

In [12]:
# ------------------ dataset ------------------------
class dataset():
    def __init__(self, shape, case, alpha, dmax):
        self.shape = shape
        self.case  = case
        self.alpha = alpha
        self.dmax = dmax

In [15]:
# -------------------- Make input ------------------------
def make_input(coor, dy, r_d, num,  angle, output_orig):
    # 200
    input_list = list()
    d_tot = coor[-1,2]
    
    curv = md.curvature_line(coor,0.01,1,500)
    for i in range(1,len(curv)):
        if curv[i,2] < curv[i-1,2] - 3:
            curv[i,2] += np.pi*2
            
    y = 0
    yd = dy
    a0 = 0
    
    while y < d_tot:
        yadd = 0
        ang = angle
        dis = np.zeros(6)
        for i in range(6):
            if y+yadd > d_tot:
                dis[i] = 1e6
                continue

            distance = md.distance_angle(curv, y + yadd, [-ang, ang])
            if distance[0] < distance[1]:
                dis[i] = - distance[0]
                
            else:
                dis[i] = distance[1]

            yadd += distance.min()
            ang *= 1.2
        

        curvature = md.curvature(coor, y, 0.01,1)[1:] # outputs aoa and curvature
        curvature[1] = curvature[1]/10 #np.sign(curvature[1]) * abs(curvature[1])**0.5 / 10
        input_list.append([y, yd*100, *curvature, *0.025/dis])#*(np.sign(dis)*abs(dis)**-0.5/10)])  #0.25 * 0.1

        # Interpolate coordinates
        dr = min( md.distance_angle(curv, y, [r_d, -r_d]) )
        if dr/2 < dy:
            yd = dr/2
        y += yd
        if dy < dr/2:
            yd = dy

    if len(input_list)< num:
        return 0
    inputs = np.array(input_list)[:num]
    for i in range(1,len(inputs)):
        if inputs[i,2] < inputs[i-1,2] - 3:
            inputs[i,2] += np.pi*2
    
    x = output_orig[:,0]
    outputs = np.interp(inputs[:,0], x, output_orig[:,1]/output_orig[0,1])
    return [inputs, outputs]

In [67]:
# ------------- gen input contant set ------------------
m = 34
n = 80
max_dist_list = [0.01, 0.02, 0.03, 0.04, 0.05]
max_ang_list = [0.05, 0.1, 0.3, 0.5]
training_list  = list()
for j in range(m):
    name = f'Data/Constant_set/S{j}'
    coor = md.load_shape(name+'/shape')   
    for max_dist in max_dist_list:
        for max_ang in max_ang_list: 
            set = dataset(j, 1, max_ang, max_dist)
            output = np.load(name+'.npy')
            data = make_input(coor, max_dist, max_ang, n, 1/4, output) #inputs -> y, dy, aoa, curv, curv1, curv2...
            if not data: continue
            set.input = data[0]
            set.output = data[1]

            training_list.append(set)

with open('training_data', 'wb') as file:
    pickle.dump(training_list, file)


In [4]:
# Check the folder
m = 20
n = 20
for j in range(m):
    for i in range(n):
        name = f'Data/Final_set/S{j}_C{i}'
        if os.path.isdir(name) and not os.path.isfile(name+'.npy'):
            print(name, 'is missing npy')
            continue
        if not os.path.isdir(name) and os.path.isfile(name+'.npy'):
            print(name+'.npy', 'is missing folder')

In [8]:
# ------------------- get flow function
def get_flow(filename):
    with open(filename+'/input_file', 'r') as file:
        content = file.read()[:50]
    ma = float(content.split()[0])
    re = float(content.split()[1])
    T0 = float(content.split()[2])
    Tw = float(content.split()[3])
    rho = float(content.split()[4])
    return [ma, re, T0, rho]

name = 'Data/Final_set/S20_C2'
get_flow(name) / np.array([10, 10000, 200, 1000])

array([0.27  , 0.8001, 1.14  , 1.001 ])

In [19]:
# ------------ gen input primary set ------------------
m = 20
n = 20
l = 80
max_dist_list = [0.01, 0.02, 0.03, 0.04, 0.05]
max_ang_list = [0.05, 0.1, 0.3, 0.5]
training_list  = list()


for j in range(m):
    for i in range(n):
        name = f'Data/Final_set/S{j}_C{i}'
        if not os.path.isdir(name): continue
        coor = md.load_shape(name+'/shape')
        
        for max_dist in max_dist_list:
            for max_ang in max_ang_list: 
                set = dataset(j, i, max_ang, max_dist)
                output = np.load(name+'.npy')
                data = make_input(coor, max_dist, max_ang, l, 1/4, output) #inputs -> y, dy, aoa, curv, curv1, curv2...
                if not data: continue
                set.flow = get_flow(name)
                set.input = data[0]
                set.output = data[1]

                training_list.append(set)

with open('training_data_primary', 'wb') as file:
    pickle.dump(training_list, file)


In [14]:
# ------------------ check folder ----------------
for j in range(20):
    for i in range(20):
        name = f'Data/Final_set/S{i}_C{j}'
        if not os.path.isdir(name): continue
        nname = name
    a = get_flow(nname)
    print(f'C{j} & {a[0]:.1f} & {a[1]/1000:.0f}000 & {a[2]:.0f}')

C0 & 5.3 & 10000 & 234
C1 & 10.3 & 11000 & 243
C2 & 2.7 & 8000 & 228
C3 & 12.2 & 19000 & 237
C4 & 10.5 & 12000 & 243
C5 & 3.6 & 10000 & 228
C6 & 5.5 & 16000 & 228
C7 & 11.8 & 13000 & 243
C8 & 10.1 & 14000 & 239
C9 & 6.6 & 14000 & 233
C10 & 4.1 & 15000 & 226
C11 & 9.4 & 15000 & 237
C12 & 6.2 & 15000 & 230
C13 & 10.1 & 19000 & 234
C14 & 6.3 & 15000 & 230
C15 & 4.6 & 14000 & 228
C16 & 7.5 & 17000 & 231
C17 & 9.8 & 17000 & 236
C18 & 3.5 & 12000 & 227
C19 & 7.1 & 18000 & 229


In [100]:
# ----------------------- New shapes and cases numbers --------------------------
m = 20
n = 30

mat = np.zeros((m,n))
for j in range(m):
    for i in range(n):
        name = f'Data/Final_set/S{j}_C{i}'
        if not os.path.isdir(name): continue
        mat[j,i] = 1

mat[4,23] = mat[6,24] = mat[15,3] = mat[8,0] = mat[19,23] = mat[9,10] = mat[5,20] = mat[12,11] = mat[13,13] = mat[18,15] = mat[2,6] = mat[10,7] = 1

for j in range(m):
    for i in range(n):
        if mat[j].sum() + mat[:,i].sum() == 1:
            print(i,j)
            mat[j,i] = 1

# Find 

16 2
22 3
25 6
26 9
27 10
28 13
29 16


In [15]:
# ------------------------ Cases as points ------------------------
points = list()
for i in range(27):
    filename = f'Cases_files/C{i}/input_file'
    with open(filename, 'r') as file:
        content = file.read()[:20]
    ma = float(content.split()[0])
    re = float(content.split()[1])
    points.append([ma, re])

np.save('casepoints.npy',np.array(points))
    